# Notebook for building and testing

In [1]:
from newsapi import NewsApiClient
from _utils import *


In [4]:
with open("credentials/api_key.txt") as f:
    api_key = f.read()

with open("credentials/mongo_client.txt") as f:
    mongo_client = f.read()

db_name = "stock_data"
collection_name = "news"

start_date = datetime(2023, 8, 1).strftime('%Y-%m-%d')
end_date = datetime(2023, 8, 31).strftime('%Y-%m-%d')
query = 'Google'

news = GetNews(api_key, query, start_date, end_date)
news_list = news.get_articles()
news.add_to_db(mongo_client, db_name, collection_name)

In [9]:
db = client[db_name]
collection = db[collection_name]

In [30]:
# find duplicates and remove them
duplicates = collection.aggregate([
    # group on title and media
    {"$group": {"_id": "$title", "count": {"$sum": 1}}},
    {"$match": {"count": {"$gt": 1}}}
])

for doc in duplicates:
    title = doc['_id']
    collection.delete_one({"title": title})

